In [1]:
from os import environ
environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ['HTTP_PROXY'] = 'http://localhost:7890'
environ['HTTPS_PROXY'] = 'http://localhost:7890'

from retrieval_qa_benchmark.datasets import MMLU
from retrieval_qa_benchmark.datasets import HotpotQA
from retrieval_qa_benchmark.datasets import WikiHop


/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = HotpotQA.build(subset='fullwiki')
dataset = MMLU.build(subset='prehistory')
# dataset[0]
dataset = WikiHop.build(subset='original')

Converting dataset...: 100%|██████████| 5129/5129 [00:00<00:00, 14557.09it/s]
